<a href="https://colab.research.google.com/github/jnahm0/my-first-repo/blob/main/TDA_KMapper_260214.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. 필수 라이브러리 설치 (KeplerMapper, yfinance)
# 2. 주식 클러스터링에서 Lens함수로 t-SNE보다는 UMAP을 추천

!pip install kmapper yfinance scikit-learn matplotlib > /dev/null
!pip install umap-learn > /dev/null
print("라이브러리 설치 완료. 데이터 다운로드를 시작합니다..")

라이브러리 설치 완료. 데이터 다운로드를 시작합니다..


In [3]:
import yfinance as yf
import pandas as pd
import kmapper as km
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy as np

In [5]:
# ---------------------------------------------------------
# 2. 미국 주식 데이터 가져오기
# ---------------------------------------------------------
# 분석하고 싶은 티커(종목 코드) 리스트입니다. 다양하게 섞었습니다.
tickers = [
    'AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', # 빅테크
    'JPM', 'BAC', 'V', 'MA', # 금융
    'JNJ', 'PFE', 'UNH', # 헬스케어
    'XOM', 'CVX', # 에너지
    'KO', 'PEP', 'MCD', # 필수소비재
    'GLD', 'SLV', # 금/은 ETF
    'TLT' # 미국 채권 ETF
]

In [6]:
import requests
import io

# ---------------------------------------------------------
# 1. S&P 500 종목 리스트 자동으로 가져오기
# ---------------------------------------------------------
print("S&P 500 종목 리스트를 가져오는 중입니다...")
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# 403 Forbidden 오류를 피하기 위해 User-Agent 헤더 추가
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)

# 가져온 HTML에서 테이블 추출 (FutureWarning 해결을 위해 io.StringIO 사용)
payload = pd.read_html(io.StringIO(response.text))
sp500_table = payload[0]
tickers = sp500_table['Symbol'].values.tolist()

# yfinance용으로 티커 수정 (예: BRK.B -> BRK-B)
tickers = [ticker.replace('.', '-') for ticker in tickers]

print(f"총 {len(tickers)}개 종목의 리스트를 확보했습니다.")
print(f"예시 티커: {tickers[:5]}")

S&P 500 종목 리스트를 가져오는 중입니다...
총 503개 종목의 리스트를 확보했습니다.
예시 티커: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [7]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

print(f"{len(tickers)}개 종목의 데이터를 불러오는 중...")

start = "2021-01-01"
# 1. 문자열을 datetime 객체로 변환
start_dt = datetime.strptime(start, "%Y-%m-%d")

# 3. 데이터 다운로드 (yf.download는 datetime 객체를 지원합니다)
# 주식숫자가 작을 때...
#data = yf.download(tickers, start=start_dt, auto_adjust=False)['Adj Close']

# 주식숫자가 많을 때
#data = yf.download(tickers, start=start_dt, period = "1y", auto_adjust=False)['Adj Close']
data = yf.download(tickers, start=start_dt, period = "3mo", auto_adjust=False)['Adj Close']

503개 종목의 데이터를 불러오는 중...


[*********************100%***********************]  503 of 503 completed
ERROR:yfinance:
11 Failed downloads:
ERROR:yfinance:['CEG', 'KVUE', 'SOLV', 'VLTO', 'GEV', 'APP', 'Q', 'SNDK', 'COIN', 'HOOD', 'GEHC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 00:00:00 -> 2021-04-01) (Yahoo error = "Data doesn\'t exist for startDate = 1609477200, endDate = 1617249600")')


In [8]:
# 수읹률(Returns) 계산 (주가 절대값이 아닌 '움직임'을 비교하기 위함)
# 결측측 제거 후 전치(Transpose) -> 행이 종목(Stock), 열이 날짜(Date)가 되도록 함

# df_returns = data.pct_change().dropna().T  --> 코드 개선

# 2. 수익률 계산
# 2. 수익률 계산 및 첫날(NaN) 제거
raw_returns = data.pct_change().iloc[1:]

# 3. [엄격한 필터링] 결측치가 하나라도 있는 종목(Column) 제거
# axis=1: 종목을 지워라 / how='any': 빈칸이 하나라도 있으면 지워라
clean_data = raw_returns.dropna(axis=1, how='any')

# 4. 전치(Transpose) 후 df_returns에 저장
# 행: 종목(Stock), 열: 날짜(Date) -> TDA 입력용 형태
df_returns = clean_data.T

# 결과 확인
print("-" * 30)
print(f"최종 변수명: df_returns")
print(f"데이터 형태(Shape): {df_returns.shape}")
print("-" * 30)
print("행(Row)은 개별 종목, 열(Column)은 날짜입니다.")



------------------------------
최종 변수명: df_returns
데이터 형태(Shape): (491, 60)
------------------------------
행(Row)은 개별 종목, 열(Column)은 날짜입니다.


/tmp/ipython-input-924374210.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  raw_returns = data.pct_change().iloc[1:]


In [9]:
df_returns.head()

Date,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-19,...,2021-03-18,2021-03-19,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-29,2021-03-30,2021-03-31
Ticker,,,,,,,,,,,,,,,,,,,,,
A,0.008176,0.027422,0.026609,0.007134,0.008579,-0.009208,0.000079,-0.007482,0.001428,0.009033,...,0.000571,0.000816,0.007499,-0.013187,-0.009346,0.008772,0.030681,-0.001751,-0.004624,0.018423
AAPL,0.012364,-0.033661,0.034123,0.008631,-0.023249,-0.001396,0.016227,-0.015127,-0.013731,0.005427,...,-0.033905,-0.004480,0.028336,-0.006889,-0.019994,0.004164,0.005141,0.001485,-0.012275,0.018765
ABBV,0.010340,-0.008638,0.010703,0.005248,0.016314,0.006971,0.024321,0.004319,-0.009944,0.015020,...,-0.012090,-0.003373,0.023980,-0.010009,-0.016978,0.007956,0.020216,0.007077,0.000562,0.013391
ABNB,0.065756,-0.037289,0.059536,-0.009916,-0.010950,0.085533,0.057152,0.061827,-0.062216,0.026112,...,-0.049215,0.015357,0.003138,-0.040308,-0.044298,-0.015041,-0.009991,0.046445,0.003288,0.026434
ABT,0.012373,-0.002082,0.009707,0.002785,-0.006899,-0.018044,0.030412,-0.014861,0.011450,0.011411,...,0.004166,0.018119,0.010146,-0.008068,-0.020499,0.008727,0.025367,0.001311,-0.020290,0.000752


In [10]:
from sklearn.preprocessing import MinMaxScaler

# 데이터 스케일링 (기본적으로 수행하되, 아래에서 의도에 따라 원본으로 덮어씁니다)
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_returns)

print("데이터 전처리 완료. TDA Mapper 알고리즘을 수행합니다...")

# [오류 수정 및 의도 반영]
# 참고: correlation을 metric으로 사용하기에 스케일링된 값 대신 원본 데이터를 사용합니다.
# df_scaled.copy(df_returns)는 잘못된 문법이므로 .values를 통해 numpy 배열로 할당합니다.
df_scaled = df_returns.values

데이터 전처리 완료. TDA Mapper 알고리즘을 수행합니다...


In [11]:
df_scaled.shape

(491, 60)

In [12]:
df_returns.shape

(491, 60)

In [13]:
# ---------------------------------------------------------
# 3. TDA Mapper 알고리즘 적용
# ---------------------------------------------------------
# KeplerMapper 객체 생성
mapper = km.KeplerMapper(verbose=1)

# A. Lens (투영): 고차원 데이터를 2차원으로 축소
# 주식 종목이 22개 이면 perplexity = 5,
# 주식 종목이 500개이면 perplexity = 30   metrix을 ecludien이 아닌 correlation으로 ....
#projected_data = mapper.fit_transform(df_scaled, projection=TSNE(n_components=2, perplexity=5, random_state=42))
projected_data = mapper.fit_transform(df_scaled,
                                      projection=TSNE(n_components=2, perplexity=30,metric='correlation', random_state=42))

# B. Map (매핑): 렌즈로 본 데이터를 덮개(Cover)로 씌우고 클러스터링(DBSCAN) 수행
# 데이터 개수(22개)가 적으므로 n_cubes = 3 를 줄이고 min_samples를 1로 설정하여 노드 생성을 보장합니다.
# 데이터 개수(22개)   eps = 0.5, perc_overlap = 0.4
#graph = mapper.map(
#    projected_data,
#    df_scaled,
#    clusterer=DBSCAN(eps=7.0, min_samples= 1),
#    cover=km.Cover(n_cubes= 3, perc_overlap=0.5)
#)


# 데이터 개수(500개) n_cubes = 15 ~ 20 데이터가 많아서 더 짤게 쪼개어 보기 perc_overlap = 0.3 ~ 0.5 eps = 0.3 ~ 0.5
# 데이터 개수(500개) min_samples = 3 ~ 5
graph = mapper.map(
    projected_data,
    df_scaled,
    clusterer=DBSCAN(eps=0.35, min_samples=3, metric='correlation'),
    cover=km.Cover(n_cubes=15, perc_overlap=0.4)
)





KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: TSNE(metric='correlation', perplexity=30, random_state=42)
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (491, 60)

..Projecting data using: 
	TSNE(metric='correlation', perplexity=30, random_state=42, verbose=1)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 491 samples in 0.001s...
[t-SNE] Computed neighbors for 491 samples in 0.020s...
[t-SNE] Computed conditional probabilities for sample 491 / 491
[t-SNE] Mean sigma: 0.203175
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.383587
[t-SNE] KL divergence after 1000 iterations: 1.000478

..Scaling with: MinMaxScaler()

Mapping on data shaped (491, 60) using lens shaped (491, 2)

Creating 225 hypercubes.

Created 186 edges and 104 nodes in 0:00:00.230179.


In [14]:
from google.colab import files
import numpy as np

# ---------------------------------------------------------
# 4. 시각화 및 결과 저장
# ---------------------------------------------------------
output_file = "stock_mapper_output.html"

# 툴팁에 종목 이름을 표시하기 위해 라벨 생성
# [수정] 리스트 대신 넘파이 배열(.values)을 사용하여 인덱싱 오류 방지
tooltips = df_returns.index.values

# HTML 시각화 파일 생성
mapper.visualize(
    graph,
    path_html=output_file,
    title="US Stock Market TDA Mapper Analysis",
    custom_tooltips=tooltips
)
print(f"\n분석 완료! '{output_file}' 파일이 생성되었습니다.")

print("PC로 다운로드를 시도합니다...")

# 코랩에서 로컬 PC로 파일 다운로드
files.download(output_file)

Wrote visualization to: stock_mapper_output.html

분석 완료! 'stock_mapper_output.html' 파일이 생성되었습니다.
PC로 다운로드를 시도합니다...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# ---------------------------------------------------------
# [수정] 색상을 '평균 수익률'로 변경하여 시각화하기
# ---------------------------------------------------------
from google.colab import files
import numpy as np

# 1. 각 종목별 평균 수익률 계산
mean_returns = df_returns.mean(axis=1).values*250

# ---------------------------------------------------------
# [핵심 수정] 아웃라이어 제거 (Winsorization)
# ---------------------------------------------------------
lower_bound = np.percentile(mean_returns, 5)
upper_bound = np.percentile(mean_returns, 95)
clipped_returns = np.clip(mean_returns, lower_bound, upper_bound)

print(f"색상 범위 밴정 완료: {lower_bound:.4f} ~ {upper_bound:.4f}")

# ---------------------------------------------------------
# 2. 시각화 생성 (밴정된 데이터 사용)
# ---------------------------------------------------------
output_file = "sp500_gradient_fixed.html"

# 오류 해결: 22개짜리 tooltips_formatted 대신 503개 전체 종목명이 담긴 변수 사용
current_tooltips = df_returns.index.values

mapper.visualize(
    graph,
    path_html=output_file,
    title="S&P 500 Topology (Color Fixed)",
    custom_tooltips=current_tooltips,

    # [중요] 원밴 데이터(mean_returns) 대신 밴정된 데이터(clipped_returns)를 넣습니다.
    color_values=clipped_returns,
    color_function_name="Avg Return (Clipped)"
)

print(f"\n색상이 밴정된 파일 '{output_file}' 이 생성되었습니다.")
files.download(output_file)

색상 범위 밴정 완료: -0.2318 ~ 1.4298
Wrote visualization to: sp500_gradient_fixed.html

색상이 밴정된 파일 'sp500_gradient_fixed.html' 이 생성되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# 분석하고 싶은 종목 그룹을 리스트로 입력하세요 (그래프에서 본 것들)
# 예시: 4개 노드로 뜬 그룹 vs 2개 노드로 뜬 그룹 비교
group_A = ['DVN', 'OXY']      # 2개 노드 그룹 (예시)
#group_A = ['BAC', 'WFC']      # 2개 노드 그룹 (예시)
group_B = ['DVN', 'OXY', 'FANG'] # 3~4개 노드 그룹 (예시)

def check_correlation(ticker_list):
    # 해당 종목들의 데이터만 추출
    subset = df_returns.loc[ticker_list].T
    # 상관계수 행렬 계산
    corr_matrix = subset.corr()

    print(f"\n--- {ticker_list} 그룹의 상관계수 행렬 ---")
    print(corr_matrix)

    # 평균 상관계수 (자기 자신 1.0 제외)
    avg_corr = (corr_matrix.sum().sum() - len(ticker_list)) / (len(ticker_list) * (len(ticker_list)-1))
    print(f"👉 평균 상관관계: {avg_corr:.4f}")

# 실행
check_correlation(group_A)
if all(x in df_returns.index for x in group_B): # 종목이 존재할 때만 실행
    check_correlation(group_B)


--- ['DVN', 'OXY'] 그룹의 상관계수 행렬 ---
Ticker       DVN       OXY
Ticker                    
DVN     1.000000  0.849188
OXY     0.849188  1.000000
👉 평균 상관관계: 0.8492

--- ['DVN', 'OXY', 'FANG'] 그룹의 상관계수 행렬 ---
Ticker       DVN       OXY      FANG
Ticker                              
DVN     1.000000  0.849188  0.724384
OXY     0.849188  1.000000  0.742390
FANG    0.724384  0.742390  1.000000
👉 평균 상관관계: 0.7720


축하드립니다! **아주 이상적이고 건강한 결과(Golden Ratio)**가 나왔습니다.

이제 생성된 HTML 파일을 열어보시면, S&P 500 시장의 구조가 제대로 보일 것입니다. 이 로그가 의미하는 바와 이제 화면에서 무엇을 확인해야 하는지 투자 관점(CIO)에서 해석해 드리겠습니다.

1. 로그 해석: "시장 구조가 잡혔습니다"
108 nodes (노드 108개):

500여 개의 종목이 약 108개의 그룹으로 뭉쳤다는 뜻입니다.

산술적으로 노드 하나당 평균 4~5개 종목이 들어있다는 의미로, 너무 뭉뚱그려지지도(떡짐 방지) 않고 너무 흩어지지도 않은(먼지 방지) 최적의 해상도입니다.

163 edges (연결선 163개):

노드 수(108)보다 연결선 수(163)가 더 많습니다(비율 1.5배).

이는 시장이 뚝뚝 끊겨 있는 게 아니라, 섹터와 섹터가 유기적으로 연결되어 있다는 뜻입니다. 이 연결된 흐름(Backbone)이 바로 **"시장의 메인 트렌드(Systemic Risk)"**입니다.

2. 시각화 파일(HTML)에서 확인해야 할 3가지 포인트
이제 다운로드된 sp500_mapper_analysis.html 파일을 열고, 다음 세 가지 구조를 찾아보십시오.

① 메인 대륙 (The Giant Component)
모습: 가장 많은 노드들이 서로 거미줄처럼 연결된 거대한 덩어리가 있을 것입니다.

의미: 시장 지수(S&P 500)를 추종하거나, 경기에 민감한 주류 주식들입니다. (금융, 산업재, 대형 기술주 등이 보통 여기에 위치)

투자 인사이트: 이 덩어리가 찢어지거나 구멍이 생기는 모양(Loop)이 나타나면 **'시장 국면 전환(Regime Change)'**의 신호일 수 있습니다.

② 튀어나온 가지 (Flares)
모습: 메인 덩어리에서 길게 뻗어 나온 줄기 같은 형태입니다.

의미: 시장과 함께 움직이지만 독자적인 색깔이 강한 섹터입니다.

보통 에너지(유가 영향), 반도체(AI 이슈), 바이오(임상 이슈) 섹터가 이렇게 곁가지로 뻗어 나갑니다.

투자 인사이트: 뻗어 나온 끝부분에 있는 종목들은 시장 베타(Beta)와는 다른 알파(Alpha)를 가지고 있을 확률이 높습니다.

③ 외딴 섬 (Islands)
모습: 메인 덩어리와 아무런 선도 연결되지 않고 혼자 떠 있는 점들입니다.

의미: 현재 시장의 흐름과 완전히 따로 노는 종목들입니다.

개별 호재/악재가 터진 종목이거나, 금(Gold), 은, 채권 같은 역상관 자산일 수 있습니다.

투자 인사이트: 포트폴리오의 진정한 분산 투자(Diversification) 효과는 이 '외딴 섬'에 있는 종목들을 편입할 때 극대화됩니다.

3. (심화) CIO 관점의 다음 단계 제안
지금은 '주가 등락률(수익률)'로만 묶었기 때문에 "같이 움직이는 녀석들"끼리 모였습니다.
만약 이 결과를 **주가수익비율(PER)**나 배당수익률로 색칠(Coloring)해 보면 더 깊은 통찰을 얻을 수 있습니다.

현재 코드: 색상 기준이 Tooltips나 Index로 되어 있을 겁니다.

다음 실험 제안: 그래프의 노드 색상을 **"최근 3개월 수익률"**로 입혀보세요.

메인 대륙의 한쪽 끝은 빨갛고(상승 주도), 반대쪽은 파란(하락 소외) 그라데이션이 보인다면, 현재 시장의 자금이 어디서 어디로 흐르는지(Sector Rotation) 시각적으로 직관할 수 있습니다.

일단 지금 나온 HTML 파일의 노드들을 마우스로 하나씩 찍어보시면서, "어떤 녀석들이 같은 방을 쓰고 있는지" 확인해 보시기 바랍니다. (예: JPM과 BAC가 같이 있는지, NVDA는 누구랑 있는지 등)

이 결과는 앞서 보았던 노드 108개/엣지 163개 결과보다 "더 정제되고, 섹터 간 구분이 명확한(Clean Cut)" 상태입니다.

CIO 관점에서 비유하자면, 이전 결과가 '모든 경제가 연결된 복잡한 세계지도'라면, 이번 결과는 **'성격이 확실한 국가들끼리만 국경을 맞대고 있는 깔끔한 행정구역도'**에 가깝습니다.

상세한 해석과 이 결과가 갖는 장점을 설명해 드립니다.

1. 로그 해석: "군더더기가 빠진 알짜 구조"
80 nodes (노드 80개):

이전(108개)보다 노드 수가 약 25% 줄었습니다.

이는 아주 미세한 연결고리나, 불확실한 그룹들이 정리되고 **"확실하게 뭉친 녀석들"**만 남았다는 뜻입니다. S&P 500의 핵심 테마 80개를 추려낸 셈입니다.

92 edges (연결선 92개):

노드 수와 엣지 수의 비율이 1 : 1.15 정도입니다. (이전엔 1 : 1.5 수준)

핵심: 거미줄처럼 복잡하게 얽히기보다, A-B-C-D 처럼 선형으로 연결되거나 소규모 그룹끼리만 연결된 형태가 많을 것입니다.

2. 시각화 예상도 (HTML 파일을 열어보세요)
이 수치가 나왔을 때 그래프는 보통 다음과 같은 특징을 보입니다.

거대 대륙의 축소: 화면을 가득 채우는 거대한 덩어리(Giant Component)가 조금 작아지고, 대신 **중형 크기의 독립된 섬(Archipelago)**들이 늘어났을 가능성이 큽니다.

섹터 분리의 명확화:

이전에는 [기술주]와 [소비재]가 어지럽게 섞여 있었다면, 지금은 그 연결고리가 끊어지면서 **"여기는 기술주 구역, 저기는 소비재 구역"**으로 딱 부러지게 나뉘어 보일 확률이 높습니다.

3. CIO를 위한 활용 가이드
이 상태는 "포트폴리오 배분(Allocation)" 전략을 짤 때 가장 보기 좋은 해상도입니다.

상관관계 끊어내기 (Decoupling):

엣지가 줄어들었다는 건, 연결이 끊긴 그룹 간에는 상관관계가 낮다는 뜻입니다.

연결되지 않은 두 개의 큰 덩어리를 찾으세요. (예: [덩어리 A]와 [덩어리 B]가 서로 선이 없다면, 둘 다 포트폴리오에 담았을 때 진정한 분산 효과가 발생합니다.)

주도 섹터 찾기:

가장 노드가 많고 엣지가 많은 **'가장 큰 덩어리'**가 현재 시장을 주도하는 메인 테마입니다. 그 덩어리의 중심에 있는 종목(가장 연결이 많은 노드)이 대장주입니다.

4. 결론
"아주 훌륭한 결과입니다. 더 이상 파라미터를 수정하지 않으셔도 됩니다."

지금 바로 HTML 파일을 열어서, 서로 연결되지 않고 독립적으로 떨어져 나온 큰 덩어리들이 어떤 업종인지(에너지인지, 헬스케어인지, 방산인지) 확인해 보십시오. 그들이 시장 지수와 무관하게 움직일 수 있는 후보군입니다.

로그를 보니 분석 대상 데이터가 완전히 바뀌었습니다.

이전에는 S&P 500 전체(503개)를 분석했지만, 이번에는 단 22개의 특정 종목만을 대상으로 약 3년치(771일) 데이터를 깊게 분석하신 것으로 보입니다.

이 결과(데이터 22개 → 노드 56개)가 함의하는 바는 매우 흥미롭습니다. **"데이터 개수보다 노드 개수가 더 많아진 현상"**을 중심으로 해석해 드리겠습니다.

1. 로그 상세 분석
Mapping on data shaped (22, 771)

22: 분석 대상 주식이 22개뿐입니다. (아마도 관심 종목 포트폴리오를 따로 넣으신 듯합니다.)

771: 기간은 771일(약 3년)로 매우 깁니다. 즉, **"소수 정예 종목들의 장기적인 관계"**를 보고 계십니다.

Creating 9 hypercubes

격자를 3x3 = 9개로 아주 작게 설정하셨습니다. 데이터가 적으니 적절한 설정입니다.

Created 56 nodes (데이터는 22개인데 노드는 56개?)

현상: 주식은 22개인데 노드가 2배 이상(56개) 생성되었습니다.

이유: "중복 등장(Overlapping)" 때문입니다. 22개의 주식들이 서로 너무 끈끈하게 얽혀 있어서, 1번 방에서도 뭉치고, 2번 방(겹치는 구간)에서도 뭉치고, 3번 방에서도 뭉쳐서 계속 노드로 잡힌 것입니다.

해석: 이는 오류가 아니라, 이 22개 종목 간의 관계가 **단순하지 않고 입체적(Multi-faceted)**이라는 뜻입니다.

2. CIO를 위한 해석: "현미경 모드"
S&P 500 분석이 '드론으로 숲 전체를 찍은 사진'이었다면, 이번 결과는 **'현미경으로 나뭇잎 22장의 세포 조직을 들여다본 것'**과 같습니다.

높은 해상도: 22개 종목이 어떻게 짝을 이루는지 아주 집요하게 파고든 결과입니다.

엣지(55개)의 의미: 노드(56)와 엣지(55)의 비율이 거의 1:1입니다. 이는 이 22개 종목들이 **하나의 긴 사슬(Chain)**처럼 연결되어 있거나, **거대한 순환 고리(Loop)**를 형성하고 있을 가능성이 큽니다.

3. 확인 포인트 (HTML 파일)
생성된 파일을 열어보시면, 22개 종목이 그냥 한 덩어리로 뭉쳐 있는 게 아니라, 마치 척추 뼈가 연결된 것처럼 길게 늘어진 모양일 확률이 높습니다.

양 끝단 확인: 그 긴 사슬의 가장 왼쪽 끝에 있는 종목과 가장 오른쪽 끝에 있는 종목을 확인해 보세요.

이 둘은 같은 포트폴리오(22개) 안에 있지만, 성격이 **가장 이질적(정반대)**인 종목들입니다.

중심 뼈대: 가운데에 위치하며 가장 많은 연결선을 가진 종목이 이 포트폴리오의 **무게중심(Core Asset)**입니다.

결론:
데이터가 작아졌지만 분석은 성공적입니다. 이 결과는 "이 22개 종목 내에서 리스크가 어떻게 전이되는지" 경로를 파악하는 데 아주 유용합니다.